In [16]:
import polars as pl
import glob

DICT_VARIABLES = {
    'ESTABLECIMIENTO_SALUD': pl.Int32,
    'GLOSA_ESTABLECIMIENTO_SALUD': pl.Categorical,
    'PERTENENCIA_ESTABLECIMIENTO_SALUD': pl.Categorical,
    'SEREMI': pl.Int8,
    'SERVICIO_DE_SALUD': pl.Int8,
    'SEXO': pl.Int8,
    'EDAD_CANT': pl.Int8,
    'TIPO_EDAD': pl.Int8,
    'EDAD_A_OS': pl.Int8,
    'PUEBLO_ORIGINARIO': pl.Int8,
    'PAIS_ORIGEN': pl.Int16,
    'GLOSA_COMUNA_RESIDENCIA': pl.Categorical,
    'REGION_RESIDENCIA': pl.Categorical,
    'GLOSA_REGION_RESIDENCIA': pl.Categorical,
    'PREVISION': pl.Int8,
    'BENEFICIARIO': pl.Categorical,
    'MODALIDAD': pl.Int8,
    'PROCEDENCIA': pl.Int8,
    'ANO_EGRESO': pl.Int16,
    'FECHA_EGRESO': pl.Date,
    'AREA_FUNCIONAL_EGRESO': pl.Int16,
    # 'DIAS_ESTADA': pl.Int16,
    'CONDICION_EGRESO': pl.Int8,
    'INTERV_Q': pl.Int8,
    'CODIGO_INTERV_Q_PPAL': pl.Int32,
}

In [39]:
with pl.StringCache():
    df = pl.read_csv('input/utf-8/*2.csv', separator=';', dtypes=DICT_VARIABLES)